
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates how to perform rolling restart in Elasticsearch using unSkript legos.</b>
</div>

<br>

<center><h2>Elasticsearch Rolling Restart</h2></center>

# Steps Overview
    1) Disable shard allocation for the node
    2) Shut down the node that needs maintenance
    3) Perform your custom configuration changes/ maintenance
    4) Start the node and allow it to join the cluster
    5) Check cluster status
    6) Re-enable shard allocation


Disabling shard allocation is simply disabling the next step in the process so that no unwanted shard allocation will occur during the shut down process of node.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import subprocess
import pprint
from pydantic import BaseModel, Field
from typing import List, Dict
from subprocess import PIPE, run
import json


from beartype import beartype
@beartype
def elasticsearch_disable_shard_allocation_printer(output):
    if output is None:
        return
    print("Shard allocations disabled for any kind shards")
    print(output)


@beartype
def elasticsearch_disable_shard_allocation(handle,
                                           host: str,
                                           api_key: str,
                                           port: int) -> Dict:
    """elasticsearch_disable_shard_allocation disallows shard allocations for any indices.

            :type handle: object
            :param handle: Object returned from Task Validate

            :type host: str
            :param host: URL of your Elasticsearch server

            :type port: int
            :param host: Port used by your Elasticsearch server

            :type api_key: str
            :param api_key: API Key for authentication of the request

            :rtype: Result Dict of result
    """
    es_path = host + ":" + str(port) + "/_cluster/settings?pretty"
    es_header = "Authorization: ApiKey" + " " + api_key
    es_dict = {"transient": {"cluster.routing.allocation.enable": "none"}}
    es_json = json.dumps(es_dict)
    cmd = ["curl", "-k", "-XPUT", "-H", "Content-Type: application/json", "-H",
           es_header,
           es_path,
           "-d",
           str(es_json)]
    try:
        raw_result = subprocess.check_output(cmd, stderr=PIPE, universal_newlines=True, shell=False)
        return raw_result
    except subprocess.CalledProcessError as e:
        return e.output


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "api_key": "api_key",
    "host": "elasticsearch_host",
    "port": "port"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(elasticsearch_disable_shard_allocation, lego_printer=elasticsearch_disable_shard_allocation_printer, hdl=hdl, args=args)

This action shuts down a single node for rolling restart.


   If you are running Elasticsearch with systemd:

    sudo systemctl stop elasticsearch.service

   If you are running Elasticsearch with SysV init:

    sudo -i service elasticsearch stop


In [37]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import pprint


from beartype import beartype
@beartype
def ssh_execute_remote_command_printer(output):
    if output is None:
        return
    print("Elasticsearch Service successfully STOPPED")
    print("\n")
    pprint.pprint(output)


@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False) -> Dict:

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)

        o = "\n".join(output)
        res[hostname] = o

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "cmd_stop_elasticsearch",
    "hosts": "host_for_ssh",
    "sudo": "run_with_sudo"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(ssh_execute_remote_command, lego_printer=ssh_execute_remote_command_printer, hdl=hdl, args=args)

In this step you can change any config that you need in the elasticsearch.yml or perform any required maintenance job. 

    Note- Please make sure that the configuration changes don't cause the failure of a node restart in the next step

This action starts the node.


   If you are running Elasticsearch with systemd:

    sudo systemctl start elasticsearch.service

   If you are running Elasticsearch with SysV init:

    sudo -i service elasticsearch start


In [32]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import pprint


from beartype import beartype
@beartype
def ssh_execute_remote_command_printer(output):
    if output is None:
        return
    print("Elasticsearch Service successfully STARTED")
    print("\n")
    pprint.pprint(output)


@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False) -> Dict:

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)

        o = "\n".join(output)
        res[hostname] = o

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "cmd_start_elasticsearch",
    "hosts": "host_for_ssh",
    "sudo": "run_with_sudo"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(ssh_execute_remote_command, lego_printer=ssh_execute_remote_command_printer, hdl=hdl, args=args)

This action checks the status of the cluster. These are the cluster statuses that you may encounter-
1. Unassigned primary shards = <span style="color:red">Red</span> Status
2. Unassigned replica shards = <span style="color:#FFBF00">Yellow</span> Status
3. All shards assigned = <span style="color:green">Green</span> Status

In [6]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import subprocess
import pprint
from pydantic import BaseModel, Field
from typing import Dict
from subprocess import PIPE
import json


from beartype import beartype
@beartype
def elasticsearch_check_health_status_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def elasticsearch_check_health_status(handle,
                                      host: str,
                                      port: int,
                                      api_key: str) -> Dict:
    """elasticsearch_check_health_status checks the status of an Elasticsearch cluster .

            :type handle: object
            :param handle: Object returned from Task Validate

            :type host: str
            :param host: URL of your Elasticsearch server

            :type port: int
            :param host: Port used by your Elasticsearch server

            :type api_key: str
            :param api_key: API Key for authentication of the request

            :rtype: Result String of result
    """
    es_path = host + ":"+str(port) + "/_cluster/health?pretty"
    es_header = "Authorization: ApiKey" + " " + api_key
    cmd = ["curl", "-k", "-XGET", "-H", es_header, es_path]
    try:
        raw_result = subprocess.check_output(cmd, stderr=PIPE, universal_newlines=True, shell=False)
        new_result = raw_result.replace("\n", "")
        result_dict = json.loads(new_result)
        d = {}
        for key in result_dict:
            d[key] = result_dict[key]
        print("The status for " + d["cluster_name"] + " is " + d["status"])
        print("\n Details -")
        return d
    except subprocess.CalledProcessError as e:
        return e.output


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "api_key": "api_key",
    "host": "elasticsearch_host",
    "port": "port"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(elasticsearch_check_health_status, lego_printer=elasticsearch_check_health_status_printer, hdl=hdl, args=args)

This action to enables shard allocation and makes the node ready to use.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import subprocess
import pprint
from pydantic import BaseModel, Field
from typing import List, Dict
from subprocess import PIPE, run
import json


from beartype import beartype
@beartype
def elasticsearch_enable_shard_allocation_printer(output):
    if output is None:
        return
    print("Shard allocations enabled for all kinds of shards")
    print(output)


@beartype
def elasticsearch_enable_shard_allocation(handle,
                                           host: str,
                                           api_key: str,
                                           port: int) -> Dict:
    """elasticsearch_enable_shard_allocation enables shard allocations for any shards for any indices.

            :type handle: object
            :param handle: Object returned from Task Validate

            :type host: str
            :param host: URL of your Elasticsearch server

            :type port: int
            :param host: Port used by your Elasticsearch server

            :type api_key: str
            :param api_key: API Key for authentication of the request

            :rtype: Result Dict of result
    """
    es_path = host + ":" + str(port) + "/_cluster/settings?pretty"
    es_header = "Authorization: ApiKey" + " " + api_key
    es_dict = {"transient": {"cluster.routing.allocation.enable": "all"}}
    es_json = json.dumps(es_dict)
    cmd = ["curl", "-k", "-XPUT", "-H", "Content-Type: application/json", "-H",
           es_header,
           es_path,
           "-d",
           str(es_json)]
    try:
        raw_result = subprocess.check_output(cmd, stderr=PIPE, universal_newlines=True, shell=False)
        return raw_result
    except subprocess.CalledProcessError as e:
        return e.output


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "api_key": "api_key",
    "host": "elasticsearch_host",
    "port": "port"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(elasticsearch_enable_shard_allocation, lego_printer=elasticsearch_enable_shard_allocation_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's Elasticsearch and SSH legos to perform rolling restart on a node in an elasticsearch cluster. To view the full platform capabilities of unSkript please visit https://unskript.com